In [1]:
import copy
import glob
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import classification_report, f1_score
    
from wrapper import XRAY
from split import split_data

image_folder = 'images'
label_path = 'Data_Entry_2017.csv'
stats_filepath = 'outputs_2.txt'
n_classes = 1 # regression problem
use_parallel = True
vision_model = torchvision.models.inception_v3()

loss_weights = torch.tensor([1.])
if torch.cuda.is_available():
    loss_weights = loss_weights.cuda()
# squared error loss
criterion = nn.MSELoss(size_average = False)
optimizer_type = torch.optim.Adam
lr_scheduler_type = optim.lr_scheduler.StepLR
num_epochs = 10
best_model_filepath = None
best_model_filepath = 'different_loss_best_model.tar'
load_model_filepath = 'best_model.tar'

# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# Since imagenet has 1000 classes, we need to change our last layer to 1 so that we get a regression problem
n_features = vision_model.fc.in_features
vision_model.fc = nn.Linear(n_features, n_classes)

cpu


In [3]:
def train_model(model, dataloaders, datasets, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    lowest_loss = 1000
    
    # list of models from all epochs
    model_list = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                    model = model.cuda()
                else:
                    inputs = Variable(inputs)
                    labels = Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                loss = criterion(outputs, labels.reshape(-1,1).float())

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]

            epoch_loss = running_loss / dataset_sizes[phase]
            
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            with open(stats_filepath, 'a') as f:
                f.write('Epoch {} {} Loss: {:.4f} \n'.format(epoch, phase, epoch_loss))

            # deep copy the model
            if phase == 'val':               
                    
                # update best model based on f1_score
                if epoch_loss < lowest_loss:
                    lowest_loss = epoch_loss
                    best_model_wts = model.state_dict()

                    state = {'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
                    if best_model_filepath is not None:
                        torch.save(state, best_model_filepath)
        
        model_list.append(copy.deepcopy(model))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(lowest_loss))
    with open(stats_filepath, 'a') as f:
        f.write('Best val loss: {:4f}\n'.format(lowest_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model_list, model

# return the scores
def evaluate_model(model, testset_loader, test_size, use_gpu):
    model.train(False)  # Set model to evaluate mode
    running_loss = 0.0
    scores = []
    # Iterate over data
    for inputs, labels in tqdm(testset_loader):
        # TODO: wrap them in Variable?
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            model = model.cuda()
        # forward
        outputs = model(inputs)
        if type(outputs) == tuple:
            outputs, _ = outputs
        scores.extend(outputs.data.tolist())
        loss = criterion(outputs, labels.reshape(-1,1).float())
        running_loss += loss.data[0]
    average_loss = running_loss / test_size
    return (average_loss, scores)

def load_saved_model(filepath, model, optimizer=None):
    state = torch.load(filepath, map_location='cpu')
    model.load_state_dict(state['state_dict'])
    # Only need to load optimizer if you are going to resume training on the model
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])

In [ ]:
train_filenames, val_filenames, test_filenames = split_data(label_path)
print('train filenames size: ', len(train_filenames))
print('validation filenames size: ', len(val_filenames))
print('test filenames size: ', len(test_filenames))

train_dataset = XRAY(image_folder, train_filenames)
val_dataset = XRAY(image_folder, val_filenames)
test_dataset = XRAY(image_folder, test_filenames)
# print([y for img, y in train_dataset])
# print([y for img, y in val_dataset])
# print([y for img, y in test_dataset])

#print out a sample image shape
image_array, label = train_dataset[4]
print(image_array.shape)
print('training dataset size: ', len(train_dataset))
print('validation dataset size: ', len(val_dataset))
print('test dataset size: ', len(test_dataset))

trainset_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=8)
valset_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, num_workers=8)
testset_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, num_workers=8)

# To view which layers are freezed and which layers are not freezed:
# for name, child in vision_model.named_children():
#     for name_2, params in child.named_parameters():
#         print(name_2, params.requires_grad)

if use_parallel:
    print("[Using all the available GPUs]")
    vision_model = nn.DataParallel(vision_model, device_ids=[0, 1])

dataloaders = {'train': trainset_loader, 'val': valset_loader}
datasets = {'train': train_dataset, 'val': val_dataset}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
optimizable_params = [param for param in vision_model.parameters() if param.requires_grad]
optimizer = optimizer_type(optimizable_params, lr=0.001)
exp_lr_scheduler = lr_scheduler_type(optimizer, step_size=7, gamma=0.1)

# If we want to load a model with saved parameters
# if load_model_filepath is not None:
#     load_saved_model(load_model_filepath, vision_model, optimizer)

In [ ]:
model_list, best_model = train_model(vision_model,
                             dataloaders,
                             datasets,
                             dataset_sizes,
                             criterion,
                             optimizer,
                             exp_lr_scheduler,
                             use_cuda,
                             num_epochs)


In [6]:
if use_parallel:
    print("[Using all the available GPUs]")
    vision_model = nn.DataParallel(vision_model, device_ids=[0, 1])

def load_saved_model(filepath, model, optimizer=None):
    state = torch.load(filepath, map_location='cpu')
    model.load_state_dict(state['state_dict'])
        
load_saved_model(load_model_filepath, vision_model)

[Using all the available GPUs]


In [ ]:
loss, scores = evaluate_model(vision_model, testset_loader, len(test_dataset), use_cuda)
loss

In [8]:
test_counts = np.array([x[1] for x in test_dataset])

In [11]:
scores = np.array(scores).reshape(-1)
stacked = np.vstack([scores, test_counts]).T
stacked
# stacked = [(i,j) for i,j in zip(scores, test_counts)]

array([[0.93267339, 0.5       ],
       [0.28071946, 0.        ],
       [0.30601984, 0.        ],
       ...,
       [0.37547657, 0.5       ],
       [0.68103075, 2.        ],
       [1.02225232, 2.        ]])

In [50]:
np.set_printoptions(precision = 3)
idx = np.argsort(stacked, axis = 0)[:,0]
ordered_filenames = np.array(test_filenames)[idx][:,0].reshape(-1,1)
print(ordered_filenames.shape)

sorted_scores = np.hstack([ordered_filenames, stacked[idx]])
print(sorted_scores.shape)
print(sorted_scores[:10])
np.savetxt('sorted_scores.csv', sorted_scores, fmt = ['%s', '%.6s', '%.4s'], delimiter=',')

(750, 1)
(750, 3)
[['00000738_000.png' '0.018727298825979233' '0.0']
 ['00000034_001.png' '0.04542520269751549' '0.0']
 ['00001280_001.png' '0.051202353090047836' '0.0']
 ['00000847_001.png' '0.057415030896663666' '0.0']
 ['00000545_005.png' '0.0626494288444519' '0.0']
 ['00000598_000.png' '0.06509900093078613' '0.0']
 ['00000545_004.png' '0.07218527793884277' '0.0']
 ['00001129_002.png' '0.08012452721595764' '0.0']
 ['00000753_001.png' '0.08622321486473083' '0.0']
 ['00001237_000.png' '0.09184719622135162' '0.0']]


In [36]:
np.array(test_filenames)[idx][:,0]
test_filenames

[('00000132_004.png', 0.5),
 ('00000214_000.png', 0.0),
 ('00000779_000.png', 0.0),
 ('00001232_004.png', 0.0),
 ('00000798_007.png', 0.0),
 ('00000344_000.png', 1.0),
 ('00000820_022.png', 0.0),
 ('00000230_004.png', 1.0),
 ('00000927_004.png', 2.0),
 ('00001003_000.png', 0.0),
 ('00000877_032.png', 0.0),
 ('00000583_000.png', 2.0),
 ('00000832_001.png', 0.0),
 ('00000506_001.png', 0.0),
 ('00000183_000.png', 2.0),
 ('00001203_008.png', 0.0),
 ('00000985_000.png', 0.0),
 ('00000050_000.png', 0.0),
 ('00001075_007.png', 2.0),
 ('00000979_003.png', 2.0),
 ('00000211_023.png', 1.0),
 ('00000167_001.png', 0.0),
 ('00001247_011.png', 2.0),
 ('00000211_016.png', 2.0),
 ('00001151_008.png', 0.0),
 ('00000583_041.png', 2.5),
 ('00001200_012.png', 2.0),
 ('00000732_001.png', 0.0),
 ('00000231_010.png', 0.0),
 ('00000796_002.png', 0.0),
 ('00000307_000.png', 3.0),
 ('00000103_006.png', 1.0),
 ('00000165_004.png', 0.0),
 ('00000289_000.png', 0.0),
 ('00001203_012.png', 0.0),
 ('00000491_018.png'

In [63]:
import pandas as pd
top_scores_counts = [x[1] for x in top_scores if x[1] >= 2]
print(len(top_scores_counts)/len(top_scores))
df = pd.DataFrame([x[1] for x in top_scores])
df.stack().value_counts().to_dict()

0.56


{0.0: 33,
 1.0: 6,
 1.3333333333333333: 2,
 1.5: 1,
 1.6666666666666667: 2,
 2.0: 35,
 2.25: 1,
 2.3333333333333335: 8,
 2.5: 5,
 3.0: 7}

In [60]:
counts = ([x[1] for x in test_dataset if x[1] >= 2])

191

In [62]:
print (len(counts) / len(test_dataset))
df = pd.DataFrame([counts])
df.stack().value_counts().to_dict()

0.25466666666666665


{2.0: 136, 2.25: 2, 2.3333333333333335: 9, 2.5: 20, 3.0: 24}

In [65]:
state = {'state_dict': vision_model.state_dict()}
torch.save(state, 'smaller_model.tar')